In [7]:
import os
import requests
import pandas as pd
from datetime import datetime
import mimetypes
import urllib3
import urllib.parse

# SSL 경고 비활성화 (개발 환경에서만 사용)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# 워드프레스 설정
WP_URL = "https://prayer-church.co.kr"  # WordPress 사이트 URL
WP_API = f"{WP_URL}/wp-json/wp/v2"
USERNAME = "prayerchurch"  # WordPress 사용자명
APP_PASSWORD = "AziD L62i Ga4b 1efW ToAs hBMB"  # 애플리케이션 비밀번호
FOLDER_PATH = "/Users/popi/Desktop/wp/files/"

def get_files_in_directory(directory_path):
    """지정된 디렉토리의 모든 파일 목록을 반환 (엑셀 파일과 숨김 파일 제외)"""
    all_files = []
    for file in os.listdir(directory_path):
        file_path = os.path.join(directory_path, file)
        # 엑셀 파일, 숨김 파일 제외
        if os.path.isfile(file_path) and not file.startswith('.') and \
           not file.endswith('.xlsx') and not file.endswith('.xls'):
            all_files.append(file_path)
    return all_files

def test_wordpress_connection():
    """워드프레스 REST API 연결 테스트"""
    print("워드프레스 연결 테스트 중...")
    
    try:
        # WP REST API 기본 엔드포인트 테스트
        response = requests.get(f"{WP_URL}/wp-json", verify=False)
        
        if response.status_code == 200:
            print("WordPress REST API 연결 성공!")
            
            # 인증 테스트
            auth = (USERNAME, APP_PASSWORD)
            auth_response = requests.get(f"{WP_API}/users/me", auth=auth, verify=False)
            
            if auth_response.status_code == 200:
                user_data = auth_response.json()
                print(f"인증 성공! 사용자: {user_data.get('name', USERNAME)}")
                return True
            else:
                print(f"인증 실패: {auth_response.status_code} - {auth_response.text}")
                return False
        else:
            print(f"WordPress REST API 연결 실패: {response.status_code}")
            return False
    except Exception as e:
        print(f"연결 테스트 중 오류 발생: {str(e)}")
        return False

def upload_media_to_wordpress(file_path):
    """워드프레스에 미디어 파일 업로드"""
    
    # 파일 이름 및 타입 확인
    orig_file_name = os.path.basename(file_path)
    
    # 한글 파일명 처리를 위해 영어로 임시 변환
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    file_ext = os.path.splitext(orig_file_name)[1]
    temp_file_name = f"upload_{timestamp}{file_ext}"
    
    content_type = mimetypes.guess_type(file_path)[0]
    if content_type is None:
        content_type = 'application/octet-stream'
    
    try:
        # 파일 데이터 읽기
        with open(file_path, 'rb') as f:
            # 인증 설정
            auth = (USERNAME, APP_PASSWORD)
            
            # Content-Disposition 헤더에서 파일명 인코딩 문제를 피하기 위해 영문 임시 파일명 사용
            headers = {
                'Content-Disposition': f'attachment; filename="{temp_file_name}"',
            }
            
            files = {'file': (temp_file_name, f, content_type)}
            
            # 미디어 업로드 API 요청
            response = requests.post(
                f"{WP_API}/media",
                auth=auth,
                headers=headers,
                files=files,
                verify=False  # SSL 인증서 검증 비활성화
            )
            
            if response.status_code in [200, 201]:
                response_data = response.json()
                
                # 업로드 성공 후 제목과 파일명을 원래 한글 이름으로 업데이트
                media_id = response_data.get('id', '')
                if media_id:
                    # 제목 업데이트 시도
                    try:
                        title_update = requests.post(
                            f"{WP_API}/media/{media_id}",
                            auth=auth,
                            json={'title': orig_file_name},
                            verify=False
                        )
                        if title_update.status_code in [200, 201]:
                            print(f"  파일명 '{orig_file_name}'(으)로 업데이트 성공!")
                        else:
                            print(f"  파일명 업데이트 실패: {title_update.status_code}")
                    except Exception as e:
                        print(f"  파일명 업데이트 중 오류: {str(e)}")
                
                return {
                    "original_file_name": orig_file_name,
                    "upload_file_name": temp_file_name,
                    "file_path": file_path,
                    "upload_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    "wordpress_url": response_data.get('source_url', ''),
                    "media_id": media_id,
                    "status": "성공"
                }
            else:
                try:
                    error_data = response.json()
                    error_msg = f"{response.status_code} - {error_data.get('message', response.text)}"
                except:
                    error_msg = f"{response.status_code} - {response.text}"
                
                return {
                    "original_file_name": orig_file_name,
                    "file_path": file_path,
                    "upload_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    "error": f"업로드 실패: {error_msg}",
                    "media_id": "",
                    "wordpress_url": "",
                    "status": "실패"
                }
    except Exception as e:
        return {
            "original_file_name": orig_file_name,
            "file_path": file_path,
            "upload_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "error": f"업로드 실패: {str(e)}",
            "media_id": "",
            "wordpress_url": "",
            "status": "실패"
        }

def save_results_to_excel(results, output_path):
    """결과를 엑셀 파일로 저장"""
    df = pd.DataFrame(results)
    
    # 칼럼 순서 정렬
    cols = ['original_file_name', 'upload_file_name', 'file_path', 'upload_date', 
            'status', 'wordpress_url', 'media_id', 'error']
    available_cols = [col for col in cols if col in df.columns]
    df = df[available_cols]
    
    df.to_excel(output_path, index=False)
    print(f"결과가 {output_path}에 저장되었습니다.")

def main():
    print(f"워드프레스 사이트: {WP_URL}")
    print(f"사용자: {USERNAME}")
    
    # 워드프레스 연결 테스트
    if not test_wordpress_connection():
        print("워드프레스 연결에 실패했습니다. 설정을 확인해 주세요.")
        return
    
    # 폴더에서 모든 파일 가져오기
    files = get_files_in_directory(FOLDER_PATH)
    print(f"\n총 {len(files)}개의 파일을 발견했습니다.")
    
    if len(files) == 0:
        print("업로드할 파일이 없습니다.")
        return
    
    # 결과를 저장할 리스트
    upload_results = []
    
    # 각 파일 업로드
    for i, file_path in enumerate(files):
        orig_file_name = os.path.basename(file_path)
        print(f"[{i+1}/{len(files)}] {orig_file_name} 업로드 중...")
        result = upload_media_to_wordpress(file_path)
        upload_results.append(result)
        print(f"  {'성공: ' + result.get('wordpress_url', '') if result.get('status') == '성공' else '실패: ' + result.get('error', '알 수 없는 오류')}")
    
    # 결과를 엑셀로 저장
    output_file = os.path.join(FOLDER_PATH, f"wordpress_upload_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx")
    save_results_to_excel(upload_results, output_file)

if __name__ == "__main__":
    main()

워드프레스 사이트: https://prayer-church.co.kr
사용자: prayerchurch
워드프레스 연결 테스트 중...
WordPress REST API 연결 성공!
인증 성공! 사용자: admin

총 5개의 파일을 발견했습니다.
[1/5] 6월 대표 기도문.txt 업로드 중...
  파일명 '6월 대표 기도문.txt'(으)로 업데이트 성공!
  성공: https://prayer-church.co.kr/wp-content/uploads/2025/05/upload_20250501172252.txt
[2/5] 6월 대표 기도문.pdf 업로드 중...
  파일명 '6월 대표 기도문.pdf'(으)로 업데이트 성공!
  성공: https://prayer-church.co.kr/wp-content/uploads/2025/05/upload_20250501172256.pdf
[3/5] 창립 주일 대표 기도문.pdf 업로드 중...
  파일명 '창립 주일 대표 기도문.pdf'(으)로 업데이트 성공!
  성공: https://prayer-church.co.kr/wp-content/uploads/2025/05/upload_20250501172301.pdf
[4/5] 주일 학교 예배 기도문.pdf 업로드 중...
  파일명 '주일 학교 예배 기도문.pdf'(으)로 업데이트 성공!
  성공: https://prayer-church.co.kr/wp-content/uploads/2025/05/upload_20250501172307.pdf
[5/5] 주일학교 헌금 기도문.pdf 업로드 중...
  파일명 '주일학교 헌금 기도문.pdf'(으)로 업데이트 성공!
  성공: https://prayer-church.co.kr/wp-content/uploads/2025/05/upload_20250501172312.pdf
결과가 /Users/popi/Desktop/wp/files/wordpress_upload_results_20250501_172317.xlsx에 저장되었습니다.
